# Maksymalny sumarycznie tydzien

In [32]:
using JuMP
using CSV
using DataFrames
using AmplNLWriter

### Prepare input data
data_mass = dropmissing(CSV.read("input_data/plecak.csv"))
data_plan = CSV.read("input_data/plan.csv")

plan_summary = by(data_plan, [:Przedmiot], plan_summary -> size(plan_summary, 1))
mass_summary = by(data_mass, [:Przedmiot], mass_summary -> sum(mass_summary[:Waga]))

przybory_mass = mass_summary[mass_summary[:Przedmiot] .== "Przybory", :][:x1] / 1e3

mass_summary = mass_summary[mass_summary[:Przedmiot] .!= "Przybory", :]

dict_plan = Dict(zip(plan_summary[:Przedmiot], plan_summary[:x1]))
dict_mass = Dict(zip(mass_summary[:Przedmiot], mass_summary[:x1]))

data_in= DataFrame(Przedmiot = String[],
    Sztuk = Int64[],
    Waga = Float64[])

for row in 1:size(plan_summary, 1)
    course = plan_summary[:Przedmiot][row]
    push!(data_in,[course, get(dict_plan, course, 0), get(dict_mass, course, 0) / 1e3])
end

### set parameters
course_no = size(data_in,1)
max_course_occurrences = maximum(data_in[:Sztuk])
days = 5
min_courses_per_day = 4
max_courses_per_day = 8
# avg mass for comparison
avg_mass = sum(data_in[:Sztuk].*data_in[:Waga]) / days + przybory_mass

# dictionaries for results
course_map = Dict(zip(1:course_no, data_in[:Przedmiot]))
mass_map = Dict(zip(1:course_no, data_in[:Waga]))

# maximum number of a course occurences per day
max_occurences_per_day  = 2
if max_occurences_per_day < max_course_occurrences
    max_course_occurrences = max_occurences_per_day
end
print(data_in)

11×3 DataFrames.DataFrame
│ Row │ Przedmiot   │ Sztuk │ Waga  │
├─────┼─────────────┼───────┼───────┤
│ 1   │ Matematyka  │ 4     │ 0.697 │
│ 2   │ Angielski   │ 3     │ 0.715 │
│ 3   │ Polski      │ 5     │ 1.676 │
│ 4   │ WF          │ 3     │ 0.297 │
│ 5   │ Religia     │ 2     │ 0.568 │
│ 6   │ Przyroda    │ 2     │ 1.214 │
│ 7   │ Technika    │ 1     │ 0.756 │
│ 8   │ Informatyka │ 1     │ 0.159 │
│ 9   │ Plastyka    │ 1     │ 0.167 │
│ 10  │ Historia    │ 1     │ 0.484 │
│ 11  │ Muzyka      │ 1     │ 0.36  │

In [33]:
### Opt start, define model, variables, constraints and objective
# Solver engine can be found on from https://ampl.com/products/solvers/open-source/ 
solver_dir="couenne-win64/couenne.exe"
mdl=Model(solver=AmplNLSolver(solver_dir))

@variable(mdl, x[1:days,1:course_no, 1:max_course_occurrences], Bin, start=1)
@variable(mdl, day[1:days])

for d in 1:days
    @constraint(mdl, day[d] == sum(sum(x[d,c,m]*data_in[:Waga][c] 
                for c in 1:course_no) 
                for m in 1:max_course_occurrences))
    
    @constraint(mdl, sum(sum(x[d,c,m]*m
                for c in 1:course_no)
                for m in 1:max_course_occurrences) <= max_courses_per_day)
    
    @constraint(mdl, sum(sum(x[d,c,m]*m
                for c in 1:course_no)
                for m in 1:max_course_occurrences) >= min_courses_per_day)
    
    for c in 1:course_no
            @constraint(mdl, sum(x[d,c,m]*m 
                    for m in 1:max_course_occurrences) <= max_course_occurrences)
    end
end

for c in 1:course_no
    @constraint(mdl, sum(sum(x[d,c,m]*m 
                for d in 1:days) 
                for m in 1:max_course_occurrences) == data_in[:Sztuk][c])
end

@objective(mdl,Max, sum((day[d]) for d in 1:days))

print(mdl)

Max day[1] + day[2] + day[3] + day[4] + day[5]
Subject to
 day[1] - 0.697 x[1,1,1] - 0.715 x[1,2,1] - 1.676 x[1,3,1] - 0.297 x[1,4,1] - 0.568 x[1,5,1] - 1.214 x[1,6,1] - 0.756 x[1,7,1] - 0.159 x[1,8,1] - 0.167 x[1,9,1] - 0.484 x[1,10,1] - 0.36 x[1,11,1] - 0.697 x[1,1,2] - 0.715 x[1,2,2] - 1.676 x[1,3,2] - 0.297 x[1,4,2] - 0.568 x[1,5,2] - 1.214 x[1,6,2] - 0.756 x[1,7,2] - 0.159 x[1,8,2] - 0.167 x[1,9,2] - 0.484 x[1,10,2] - 0.36 x[1,11,2] == 0
 x[1,1,1] + x[1,2,1] + x[1,3,1] + x[1,4,1] + x[1,5,1] + x[1,6,1] + x[1,7,1] + x[1,8,1] + x[1,9,1] + x[1,10,1] + x[1,11,1] + 2 x[1,1,2] + 2 x[1,2,2] + 2 x[1,3,2] + 2 x[1,4,2] + 2 x[1,5,2] + 2 x[1,6,2] + 2 x[1,7,2] + 2 x[1,8,2] + 2 x[1,9,2] + 2 x[1,10,2] + 2 x[1,11,2] <= 8
 x[1,1,1] + x[1,2,1] + x[1,3,1] + x[1,4,1] + x[1,5,1] + x[1,6,1] + x[1,7,1] + x[1,8,1] + x[1,9,1] + x[1,10,1] + x[1,11,1] + 2 x[1,1,2] + 2 x[1,2,2] + 2 x[1,3,2] + 2 x[1,4,2] + 2 x[1,5,2] + 2 x[1,6,2] + 2 x[1,7,2] + 2 x[1,8,2] + 2 x[1,9,2] + 2 x[1,10,2] + 2 x[1,11,2] >= 4
 x[1,1,1]

In [34]:
# solve
status = solve(mdl)

Couenne 0.5.6 -- an Open-Source solver for Mixed Integer Nonlinear Optimization
Mailing list: couenne@list.coin-or.org
Instructions: http://www.coin-or.org/Couenne
couenne: 
ANALYSIS TEST: Loaded instance "C:\Users\bialekj\.julia\v0.6\AmplNLWriter\.solverdata\jl_248C.tmp.nl"
Constraints:           81
Variables:            115 (110 integer)
Auxiliaries:           22 (21 integer)

Coin0506I Presolve 46 (-31) rows, 85 (-52) columns and 230 (-206) elements
Clp0006I 0  Obj 0 Primal inf 43.999984 (16) Dual inf 61.299915 (85)
Clp0006I 31  Obj -22.866 Primal inf 35.499986 (14)
Clp0006I 62  Obj -19.694
Clp0000I Optimal - objective value -19.694
Clp0032I Optimal objective -19.694 - 62 iterations time 0.002, Presolve 0.00
Clp0000I Optimal - objective value -19.694
Couenne: new cutoff value -1.9694000000e+001 (0.013 seconds)
NLP Heuristic: NLP0014I             2         OPT -19.694       13 0.017
no solution.
Cbc0004I Integer solution of -19.694 found after 0 iterations and 0 nodes (0.02 seconds)


:Optimal

In [35]:
### print and save results
day_of_week=Dict(zip([1, 2, 3, 4, 5], ["Poniedzialek", "Wtorek", "Sroda", "Czwartek", "Piatek"]))

plan = DataFrame([String,String,String,String,String],
    [:Poniedzialek,:Wtorek,:Sroda,:Czwartek,:Piatek], 10)

for d in 1:days
    i=1
    plan[i,d] = string(getvalue(day[d])+ przybory_mass)
    i+=1
    for c in 1:course_no
        for m in 1:max_course_occurrences
            value = getvalue(x[d,c,m])*m
            if abs(value-0.0)>1e-3              
                for repeat in 1:m
                    plan[i,d] = get(course_map, c, "brak kursu w slowniku")
                    i+=1
                end
            end
        end
    end
    for rest in i:10
         plan[rest,d] = ""
    end
end

max_mass = maximum(getvalue(day)) + przybory_mass


println("masa maxymalna")
println(max_mass)
println("masa srednia bez optim")
println(avg_mass)
print(plan)

CSV.write("results/plan_wynik_max_summ.csv",plan)

masa maxymalna
[7.116]
masa srednia bez optim
[5.3538]
10×5 DataFrames.DataFrame
│ Row │ Poniedzialek │ Wtorek     │ Sroda     │ Czwartek   │ Piatek      │
├─────┼──────────────┼────────────┼───────────┼────────────┼─────────────┤
│ 1   │ [5.466]      │ [4.653]    │ [4.671]   │ [4.863]    │ [7.116]     │
│ 2   │ Matematyka   │ Matematyka │ Angielski │ Matematyka │ Matematyka  │
│ 3   │ Polski       │ Polski     │ Polski    │ Angielski  │ Angielski   │
│ 4   │ WF           │ WF         │ WF        │ Polski     │ Polski      │
│ 5   │ Przyroda     │ Religia    │ Religia   │ Muzyka     │ Przyroda    │
│ 6   │ Plastyka     │            │           │            │ Technika    │
│ 7   │              │            │           │            │ Informatyka │
│ 8   │              │            │           │            │ Historia    │
│ 9   │              │            │           │            │             │
│ 10  │              │            │           │            │             │

CSV.Sink{Void,DataType}(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        missingstring: ""
        dateformat: nothing
        decimal: '.'
        truestring: 'true'
        falsestring: 'false'
        internstrings: true, IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "results/plan_wynik_max_summ.csv", 42, true, String["Poniedzialek", "Wtorek", "Sroda", "Czwartek", "Piatek"], 5, false, Val{false})